**Cell 1 :Installing the basic libraries.**

In [1]:
!pip install transformers datasets accelerate evaluate --quiet
!pip install wandb --quiet
!pip install rouge_score --quiet
!pip install torch torchvision torchaudio --quiet
!pip install sentencepiece --quiet
!pip install fastai timm peft --quiet
!pip install numpy pandas matplotlib tqdm scipy flask fastapi gradio streamlit --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 125.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

**Cell 2 : Import necessary libraries**


In [2]:
import torch
from transformers import BartTokenizer, BartForConditionalGeneration
from datasets import load_dataset
import numpy as np
import pandas as pd


**Cell 3 :Load the CNN/DailyMail Dataset**

In [3]:
# Install datasets
!pip install datasets --quiet

# Load the CNN/DailyMail dataset (Subset of 500 samples)
dataset = load_dataset("cnn_dailymail", "3.0.0", split="train[:500]")
print(f"Dataset contains {len(dataset)} samples.")



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

...

test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Dataset contains 500 samples.


**Cell 4: this cell is to check for long articles and see if we need to handle them**\

---
The BART model has a maximum token limit of 1024 for summarization.
Articles longer than 1024 tokens need to be processed differently (either truncated or chunked). That why we need to know if we have large articles and how many.


In [4]:
# Install Transformers if needed
!pip install transformers --quiet

# Initialize the tokenizer
from transformers import BartTokenizer
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")

# Analyze the article lengths
def count_tokens(text):
    tokens = tokenizer.encode(text, truncation=False)
    return len(tokens)

article_lengths = [count_tokens(article) for article in dataset["article"]]

print(f"Average Length: {np.mean(article_lengths)} tokens")
print(f"Maximum Length: {np.max(article_lengths)} tokens")
print(f"Number of Articles > 1024 tokens: {sum(1 for l in article_lengths if l > 1024)}")


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

Average Length: 756.882 tokens
Maximum Length: 2225 tokens
Number of Articles > 1024 tokens: 108


In [5]:
import nltk
nltk.download('all')
from transformers import BartTokenizer

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_rus to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |  

**Cell 5 :sentence-based chunking, tokenization, and padding of each article**

---
Cell 5 handles the sentence-based chunking, tokenization, and padding of each article. First, it splits the article into individual sentences. These sentences are grouped into chunks that do not exceed 1024 tokens, which is the maximum input size for the BART model. If the last chunk is shorter than 1024 tokens, it is padded with zeros to match the required length. At the same time, an attention mask is created where 1 represents a real token and 0 represents padding. Finally, the cell outputs two lists: one for the tokenized chunks and one for their attention masks. This process ensures that each chunk is ready for the model's input.


In [6]:
# Define the function for sentence-based chunking (Fixed Version)
def sentence_based_chunking(article, max_length=1024):
    """
    Splits the article into sentences, creates chunks of sentences
    that do not exceed 1024 tokens, and prepares them for tokenization.

    Args:
    - article (str): The full article text.
    - max_length (int): Maximum number of tokens per chunk.

    Returns:
    - chunks (list of lists): Tokenized chunks padded to max_length.
    - attention_masks (list of lists): Attention masks for each chunk.
    """
    # Split the article into sentences
    sentences = nltk.sent_tokenize(article)

    # Initialize variables
    chunks = []
    attention_masks = []
    current_chunk = []
    current_length = 0

    # Loop through each sentence
    for sentence in sentences:
        # Tokenize the sentence to count its length
        tokenized_sentence = tokenizer.encode(sentence, add_special_tokens=False)
        sentence_length = len(tokenized_sentence)

        # If adding the sentence exceeds the max length, start a new chunk
        if current_length + sentence_length > max_length:
            # Pad the chunk to 1024 tokens
            padding_length = max_length - current_length
            current_chunk.extend([0] * padding_length)

            # Enforce the attention mask to be exactly 1024
            attention_mask = [1] * (max_length - padding_length) + [0] * padding_length

            # Save the chunk and reset
            chunks.append(current_chunk)
            attention_masks.append(attention_mask)
            current_chunk = tokenized_sentence
            current_length = sentence_length
        else:
            # Add the sentence to the current chunk
            current_chunk.extend(tokenized_sentence)
            current_length += sentence_length

    # Handle the last chunk if it is not empty
    if current_chunk:
        # Pad the last chunk
        padding_length = max_length - len(current_chunk)
        current_chunk.extend([0] * padding_length)

        #  Enforce the attention mask to be exactly 1024
        attention_mask = [1] * (max_length - padding_length) + [0] * padding_length

        # Append the final chunk
        chunks.append(current_chunk)
        attention_masks.append(attention_mask)

    return chunks, attention_masks


**Cell 6:Full Dataset Chunking and DataLoader Initialization**

---
Cell 6 processes all the articles from the dataset by applying the sentence-based chunking function from Cell 5. It loops through each article, splits it into fixed-size chunks of 1024 tokens, and creates attention masks. All the chunks and their masks are collected into lists, which are then converted into PyTorch tensors for efficient processing. These tensors are combined into a TensorDataset, and a DataLoader is initialized with a batch size of 2. This allows the data to be easily fed into the model in batches, ensuring optimal memory usage and faster processing during summarization.











In [7]:
from torch.utils.data import DataLoader, TensorDataset
import torch

# Initialize empty lists to hold all chunks and attention masks
all_chunks = []
all_attention_masks = []

# Loop through each article and apply the sentence-based chunking
for article in dataset['article']:
    chunks, attention_masks = sentence_based_chunking(article)
    all_chunks.extend(chunks)
    all_attention_masks.extend(attention_masks)

# Convert lists to PyTorch tensors
input_ids = torch.tensor(all_chunks)
attention_masks = torch.tensor(all_attention_masks)

# Create a TensorDataset
dataset = TensorDataset(input_ids, attention_masks)

# Initialize DataLoader for batching
BATCH_SIZE = 2
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE)

# Display information
print(f"Total Chunks: {len(all_chunks)}")
print(f"Total Batches: {len(dataloader)}")
print(f"First Batch Input IDs Shape: {next(iter(dataloader))[0].shape}")
print(f"First Batch Attention Mask Shape: {next(iter(dataloader))[1].shape}")

Total Chunks: 610
Total Batches: 305
First Batch Input IDs Shape: torch.Size([2, 1024])
First Batch Attention Mask Shape: torch.Size([2, 1024])


**Cell 7: Display the Tokenized Data**

In [8]:
# Display the first tokenized example
print("Tokenized Article:\n", all_chunks[0])
print("\nAttention Mask:\n", all_attention_masks[0])


Tokenized Article:
 [574, 4524, 6, 1156, 36, 1251, 43, 480, 3268, 10997, 999, 3028, 7312, 20152, 3077, 899, 7, 10, 431, 984, 844, 153, 1358, 4006, 4, 134, 153, 43, 13016, 25, 37, 4072, 504, 15, 302, 6, 53, 37, 9838, 5, 418, 351, 75, 2471, 10, 8921, 15, 123, 4, 18322, 7312, 20152, 25, 3268, 10997, 11, 22, 29345, 10997, 8, 5, 9729, 9, 5, 5524, 113, 598, 5, 10208, 9, 20445, 6730, 1952, 198, 5, 232, 6, 5, 664, 2701, 161, 37, 34, 117, 708, 7, 856, 3961, 1334, 39, 1055, 409, 15, 1769, 1677, 6, 4076, 8, 6794, 1799, 4, 113, 100, 218, 75, 563, 7, 28, 65, 9, 167, 82, 54, 6, 25, 1010, 25, 51, 1004, 504, 6, 6017, 907, 1235, 10, 2232, 1612, 512, 2783, 50, 402, 1122, 60, 37, 174, 41, 2059, 33242, 656, 42, 353, 4, 113, 100, 218, 75, 206, 38, 581, 28, 1605, 31879, 4, 113, 133, 383, 38, 101, 2159, 32, 383, 14, 701, 59, 158, 2697, 480, 2799, 8, 32570, 8, 37206, 72, 3750, 504, 6, 7312, 20152, 40, 28, 441, 7, 23104, 11, 10, 10297, 6, 907, 10, 4076, 11, 10, 8881, 50, 192, 5, 8444, 822, 22, 40534, 523, 35, 

**Cell 8: Load the Pre-trained BART Model**

In [9]:
# Initialize the model for summarization
from transformers import BartForConditionalGeneration

# Load the pre-trained BART model
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
model.to('cuda')  # Use GPU if available

# List to hold all generated summaries
generated_summaries = []

# Loop through each batch in the DataLoader
print(" Generating summaries for each batch...")
for batch in dataloader:
    # Unpack the batch and move to GPU if available
    input_ids, attention_mask = batch
    input_ids = input_ids.to('cuda')
    attention_mask = attention_mask.to('cuda')

    # Generate summaries with the model
    summary_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        num_beams=4,           # Beam search for better quality
        max_length=150,        # Maximum length of summary
        min_length=30,         # Minimum length of summary
        early_stopping=True,   # Stop early if all beams match
        no_repeat_ngram_size=2 # Avoid repetition
    )

    # Decode the summaries and add to the list
    summaries = [tokenizer.decode(g, skip_special_tokens=True) for g in summary_ids]
    generated_summaries.extend(summaries)

print(f" Summarization complete. Total summaries generated: {len(generated_summaries)}")


model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

 Generating summaries for each batch...
 Summarization complete. Total summaries generated: 610


**Cell 9: Display Summaries and Evaluate**

---
Cell 9 is responsible for displaying a few random summaries generated by the BART model, alongside their original article chunks. It selects 3 random samples from the generated_summaries list and the corresponding all_chunks list. The original chunk is decoded back into readable text using the tokenizer, and it is printed along with its generated summary. This allows for quick visual inspection to evaluate the quality and accuracy of the summarization process. The separator lines (---) are added to make it easier to distinguish between different samples.

The output helps us understand:

If the summaries are accurate and well-formed.

If important information is preserved in the summarization.

If there are any obvious issues like incomplete sentences or repetition.


In [10]:
# Display and evaluate the generated summaries
import random

# Number of samples to display
num_samples = 3

# Display a few random samples
print(" Displaying a few random summaries:")
for _ in range(num_samples):
    # Select a random index
    idx = random.randint(0, len(generated_summaries) - 1)

    # Display the original article
    print(f"\n🔹 **Original Article (Chunk {idx}):**")
    original_text = tokenizer.decode(all_chunks[idx], skip_special_tokens=True)
    print(original_text[:700], "...")  # Display first 700 characters for readability

    # Display the generated summary
    print(f"\n🔹 **Generated Summary (Chunk {idx}):**")
    print(generated_summaries[idx])
    print("-" * 100)


 Displaying a few random summaries:

🔹 **Original Article (Chunk 295):**
KABUL, Afghanistan (CNN) -- Australia's new prime minister assured Afghan President Hamid Karzai on Saturday that his country's troops intend to be in Afghanistan "for the long haul."French President Nicolas Sarkozy, left, shakes hands with Afghan President Hamid Karzai in Kabul.Kevin Rudd and French President Nicolas Sarkozy flew into the Afghan capital, Kabul, on Saturday to meet with Karzai and visit their respective country's troops participating in the NATO-led International Security Assistance Force.In a morning meeting, Sarkozy "assured President Karzai of his government's long term political and military support with the people of Afghanistan," according to a statement from the Afgha ...

🔹 **Generated Summary (Chunk 295):**
Australian Prime Minister Kevin Rudd meets with Afghan President Hamid Karzai. Rudd: "We will be, as I said before, in this country, Afghanistan, for the long haul" Rudd and French Pre

**Cell 10: ROUGE Evaluation**

---
Cell 10 is responsible for evaluating the quality of the generated summaries using the ROUGE metric. ROUGE measures the overlap between the generated summaries and the original text in terms of:

ROUGE-1: Measures the overlap of individual words (unigrams).

ROUGE-2: Measures the overlap of consecutive word pairs (bigrams).

ROUGE-L: Measures the longest matching sequence of words (longest common subsequence).

In the cell, each generated summary is compared to its original chunk using these metrics. The scores are collected for each summary, and at the end, the average scores are calculated and displayed. These averages provide a sense of how well the summaries capture the main content and structure of the original articles.

The results indicate the quality and accuracy of the summarization. Higher scores mean better preservation of important information and context.


In [11]:
# Install rouge-score if not already installed
!pip install rouge-score --quiet

from rouge_score import rouge_scorer
import numpy as np

# Initialize the ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Lists to store the individual scores
rouge1_scores = []
rouge2_scores = []
rougeL_scores = []

# Loop through each chunk and its generated summary
print(" Evaluating ROUGE scores for each summary...")
for idx, summary in enumerate(generated_summaries):
    # Decode the original chunk back to text
    original_text = tokenizer.decode(all_chunks[idx], skip_special_tokens=True)

    # Compute the ROUGE scores
    scores = scorer.score(original_text, summary)

    # Collect the scores
    rouge1_scores.append(scores['rouge1'].fmeasure)
    rouge2_scores.append(scores['rouge2'].fmeasure)
    rougeL_scores.append(scores['rougeL'].fmeasure)

# Compute the average scores
avg_rouge1 = np.mean(rouge1_scores)
avg_rouge2 = np.mean(rouge2_scores)
avg_rougeL = np.mean(rougeL_scores)

# Display the results
print("\n **ROUGE Evaluation Complete:**")
print(f"ROUGE-1 (Unigram Overlap): {avg_rouge1:.4f}")
print(f"ROUGE-2 (Bigram Overlap): {avg_rouge2:.4f}")
print(f"ROUGE-L (Longest Common Subsequence): {avg_rougeL:.4f}")


 Evaluating ROUGE scores for each summary...

 **ROUGE Evaluation Complete:**
ROUGE-1 (Unigram Overlap): 0.2127
ROUGE-2 (Bigram Overlap): 0.1788
ROUGE-L (Longest Common Subsequence): 0.1902


**Cell 11: Combine Summaries for Full Articles**

---
Cell 11 combines the individual chunked summaries back into full article summaries. It loops through the count of chunks for each original article, retrieves the corresponding summaries from the generated_summaries list, joins them into a single text, and stores them in combined_summaries. The final output confirms that the number of combined summaries matches the original number of articles, ensuring alignment for evaluation.


In [18]:
from datasets import load_dataset

# Step 1: Re-load the original articles (to maintain the order)
print(" Re-loading original articles from HuggingFace dataset...")
original_dataset = load_dataset("cnn_dailymail", "3.0.0", split="train[:500]")
original_articles = original_dataset['article']
print(f" Number of original articles: {len(original_articles)}")

# Step 2: Initialize counters and lists
current_index = 0
combined_summaries = []
summarization_counts = []

print("\n Counting chunks for each article...")

# Step 3: Re-create summarization counts by chunking the original articles
for article in original_articles:
    chunks, _ = sentence_based_chunking(article)
    summarization_counts.append(len(chunks))

print(f" Number of articles: {len(summarization_counts)}")
print(f" Total chunks across articles: {sum(summarization_counts)}")

print("\n Combining summaries for full articles...")

# Step 4: Loop through each article count and merge its chunks
for chunk_count in summarization_counts:
    # Get the summaries for this specific article's chunks
    article_chunks = generated_summaries[current_index:current_index + chunk_count]

    # Join them into one full article summary
    combined_summary = " ".join(article_chunks)

    # Append to the combined summaries list
    combined_summaries.append(combined_summary)

    # Update the index to move to the next article
    current_index += chunk_count

# Step 5: Display final status
print(f"\n Combined summaries for {len(combined_summaries)} articles.")


 Re-loading original articles from HuggingFace dataset...
 Number of original articles: 500

 Counting chunks for each article...
 Number of articles: 500
 Total chunks across articles: 610

 Combining summaries for full articles...

 Combined summaries for 500 articles.


**Cell 12: Merging Chunked Summaries Back into Full Article Summaries**

---

This code first creates a mapping of each original article to its corresponding chunk indices, ensuring that each chunk is linked back to its parent article. It then iterates through the original 500 articles, retrieves the summaries for all its chunks, merges them into a single coherent summary, and stores them in combined_summaries. Finally, it confirms that the total number of combined summaries matches the original article count (500), indicating successful reconstruction.

In [19]:
from collections import defaultdict

# Step 1: Create a mapping of original articles to their chunk indices
article_mapping = defaultdict(list)

# Re-load the original dataset (to maintain the order)
original_dataset = load_dataset("cnn_dailymail", "3.0.0", split="train[:500]")
original_articles = original_dataset['article']

# Counter to track chunk index
chunk_counter = 0

for idx, article in enumerate(original_articles):
    # Tokenize and chunk the article
    chunks, _ = sentence_based_chunking(article)
    # Map each chunk to its original article index
    for _ in chunks:
        article_mapping[idx].append(chunk_counter)
        chunk_counter += 1

# Step 2: Combine the summaries for each original article
combined_summaries = []
for i in range(500):  # Loop through each original article
    article_chunks = article_mapping[i]
    # Merge the summaries for the chunks belonging to the same article
    merged_summary = " ".join(generated_summaries[j] for j in article_chunks)
    combined_summaries.append(merged_summary)

# Final check
print(f"Number of combined summaries after merging: {len(combined_summaries)}")


Number of combined summaries after merging: 500


**Cell 13: ROUGE Evaluation of Combined Summaries**

---
This code evaluates the quality of the combined summaries by comparing them to the original articles using the ROUGE metric, which measures word and phrase overlap. It re-loads the original 500 articles, calculates ROUGE-1, ROUGE-2, and ROUGE-L scores for each generated summary, and stores the results in a list. Finally, it displays a few random examples, showing the original article alongside its generated summary for quick inspection of the summarization quality. This helps to assess how much important content was preserved in the summarization process.


In [20]:
import random
from rouge_score import rouge_scorer

# Initialize ROUGE evaluator
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Re-load the original articles
print(" Re-loading original articles from HuggingFace dataset...")
from datasets import load_dataset
original_dataset = load_dataset("cnn_dailymail", "3.0.0", split="train[:500]")
original_articles = original_dataset['article']

# Lists to hold evaluation results
rouge_scores = []

print(" Evaluating combined summaries...")

# Loop through each original article and its combined summary
for original, summary in zip(original_articles, combined_summaries):
    # Calculate ROUGE scores
    scores = scorer.score(original, summary)
    rouge_scores.append(scores)

print("\n Evaluation Complete!")

# Display a few random samples (using the minimum length to avoid index issues)
min_length = min(len(original_articles), len(combined_summaries))

print("\n💡 Displaying Random Summaries:")
for i in random.sample(range(min_length), 3):
    print(f"\n🔹 **Original Article (Index {i}):**\n{original_articles[i][:500]}...")
    print(f"\n🔹 **Generated Summary (Index {i}):**\n{combined_summaries[i]}")
    print("-" * 100)


 Re-loading original articles from HuggingFace dataset...
 Evaluating combined summaries...

 Evaluation Complete!

💡 Displaying Random Summaries:

🔹 **Original Article (Index 220):**
ATLANTA, Georgia (CNN) -- The Rev. James Orange, a civil rights activist whose 1965 jailing sparked a fatal protest that ultimately led to the famed Selma-to-Montgomery march and the Voting Rights Act, died Saturday at Atlanta's Crawford Long Hospital, the Southern Christian Leadership Conference said in a statement. He was 65. Orange was a native of Birmingham, Alabama, "who resided in southwest Atlanta for four decades while fighting the good fight for equality and social justice for all manki...

🔹 **Generated Summary (Index 220):**
The Rev. James Orange died Saturday at Atlanta's Crawford Long Hospital. Orange's 1965 jailing sparked a fatal protest that ultimately led to the Selma-to-Montgomery march. He was a project coordinator at the Southern Christian Leadership Conference.
-----------------------

**Cell :14 Calculate average ROUGE scores**

---
This code calculates the average ROUGE scores across all evaluated summaries. It extracts the F1 scores for ROUGE-1, ROUGE-2, and ROUGE-L from each entry in rouge_scores, sums them up, and divides by the total number of summaries to get the average. Finally, it displays the average ROUGE scores in a readable format, indicating how well the generated summaries preserved important content and structure from the original articles.


In [21]:
# Calculate average ROUGE scores
average_rouge1 = sum([score['rouge1'].fmeasure for score in rouge_scores]) / len(rouge_scores)
average_rouge2 = sum([score['rouge2'].fmeasure for score in rouge_scores]) / len(rouge_scores)
average_rougeL = sum([score['rougeL'].fmeasure for score in rouge_scores]) / len(rouge_scores)

print(f"\n Average ROUGE Scores:")
print(f"ROUGE-1 (F1 Score): {average_rouge1:.4f}")
print(f"ROUGE-2 (F1 Score): {average_rouge2:.4f}")
print(f"ROUGE-L (F1 Score): {average_rougeL:.4f}")



 Average ROUGE Scores:
ROUGE-1 (F1 Score): 0.1827
ROUGE-2 (F1 Score): 0.1488
ROUGE-L (F1 Score): 0.1619


**Cell 15: Save the Model**

In [26]:
import shutil

# Compress the model folder into a zip file
shutil.make_archive('bart_summarization_model', 'zip', './bart_summarization_model')
print("✅ Folder successfully compressed to bart_summarization_model.zip")


✅ Folder successfully compressed to bart_summarization_model.zip


In [27]:
from IPython.display import FileLink

# Generate a download link
FileLink(r'bart_summarization_model.zip')


/content/bart_summarization_model.zip

In [28]:
from google.colab import files

# Download the zip file directly to your computer
files.download('bart_summarization_model.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>